In [1]:
# pip install pandas
# pip install sodapy

import pandas as pd
import numpy as np
from sodapy import Socrata 
import pickle


# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.seattle.gov", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.seattle.gov',
                 '6dqBidzTCT6os6H6kLyyh54u4',
                 username="panherbert@gmail.com",
                 password="Aa760520")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results_2018 = client.get("6yaw-2m8q", select='occupancydatetime, sum(case(paidoccupancy > parkingspacecount, parkingspacecount, paidoccupancy <= parkingspacecount,  paidoccupancy)) as adjpaidoccupancy, sum(parkingspacecount) as parkingspacecnt', where=("blockfacename == 'UNIVERSITY ST BETWEEN ALASKAN E RDWY WAY AND WESTERN AVE'"), group='occupancydatetime',order='occupancydatetime ASC',limit=999999999)
results_2019 = client.get("qktt-2bsy", select='occupancydatetime, sum(case(paidoccupancy > parkingspacecount, parkingspacecount, paidoccupancy <= parkingspacecount,  paidoccupancy)) as adjpaidoccupancy, sum(parkingspacecount) as parkingspacecnt', where=("blockfacename == 'UNIVERSITY ST BETWEEN ALASKAN E RDWY WAY AND WESTERN AVE'"), group='occupancydatetime',order='occupancydatetime ASC',limit=999999999)

In [2]:
# Convert data to pandas DataFrame
results_df = pd.DataFrame.from_records((results_2018+results_2019))

In [3]:
# Converting 'occupancydatetime' to datetime
results_df.occupancydatetime = pd.to_datetime(results_df.occupancydatetime)

# Converting 'adjpaidoccupancy', 'parkingspacecnt' to int
results_df['adjpaidoccupancy'] = results_df['adjpaidoccupancy'].apply(pd.to_numeric)
results_df['parkingspacecnt'] = results_df['parkingspacecnt'].apply(pd.to_numeric)



# # Reorganizing column orders
# results_df = results_df['adjpaidoccupancy',  ]

In [4]:
# Calculating avaiable parking space
results_df['availableparkingspace'] = results_df.parkingspacecnt - results_df.adjpaidoccupancy

In [5]:
results_df[['occupancydatetime', 'availableparkingspace']].head()

,occupancydatetime,availableparkingspace
0,2018-01-02 08:00:00,5
1,2018-01-02 08:01:00,5
2,2018-01-02 08:02:00,5
3,2018-01-02 08:03:00,5
4,2018-01-02 08:04:00,5


In [7]:
# Pickling out dataframe
with open('dateframe_pickle','wb') as file:
    pickle.dump(results_df, file)